<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/220516_Gradient_AutoDifferentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## 그래디언트 계산하기
- 자동 미분을 위해 tf는 정방향 패스 동안 연산의 순서를 기억하고, 역방향 패스 동안 이 목록을 역순으로 이동하여 그래디언트를 계산한다.
- 텐서플로우는 이를 위한 `tf.GradientTape` API를 제공한다. 이는 컨텍스트 안에서 실행된 모든 연산을 테이프에 기록한 뒤 후진 방식 자동 미분(Reverse Mode Differentiation)을 통해 기록된 연산의 그래디언트를 계산한다.

```python
with tf.GradientTape() as tape:
  b = a**2 # 여기서 a와 b가 기록되는 것 같고

db_da = tape.gradient(b, a) # 여기서 기록된 값들로 gradient를 구하는 것 같다.
```

In [ ]:
x = tf.Variable(3.)

with tf.GradientTape() as tape:
  y = x**2

dy_dx = tape.gradient(y, x) # gradient(a, b) = da / db -> 즉 이 연산은 2 * 3 = 6
dy_dx.numpy()

### 스칼라 외에도 모든 텐서에 대해 작동한다.

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name = 'w')
b = tf.Variable(tf.zeros(2, dtype = tf.float32), name = 'b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent = True) as tape:
  y = x @ w + b # @ : matmul
  loss = tf.reduce_mean(y**2) # y**2 값 전체를 1개의 평균으로 냄 : 일단 변수로 갖고 있는 듯

[dl_dw, dl_db] = tape.gradient(loss, [w, b]) 

In [ ]:
print(loss) 
print(w.shape)
print(dl_dw.shape)

In [ ]:
print(dl_dw, dl_db)

In [ ]:
my_vars = {
    'w' : w,
    'b' : b
}

grad = tape.gradient(loss, my_vars) # 위와 동일한데 그 형태만 dict로 전달한 거임
grad['b']

## 모델에 대한 그래디언트
- `tf.Varaibles`를 `tf.Module` 또는 해당 서브클래스 `layers.Layer` or `keras.Model` 중 하나로 수집한다.
- `tf.Module`의 모든 서브클래스는 `Module.trainable_variables`로 변수를 집계하므로 그래디언트를 쉽게 계산할 수 있다.

In [ ]:
layer = tf.keras.layers.Dense(2, activation = 'relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # 정방향 연산 (forward pass)
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# 그래디언트 계산 - trainable_variables로 편안하게 모든 가중치와 바이어스를 지정했다.
grad = tape.gradient(loss, layer.trainable_variables)

for var, g in zip(layer.trainable_variables, grad):
  print(f"{var.name}, shape : {g.shape}")

## 테이프의 감시 대상 제어
- 기본 : `tf.variable`에 액세스 후 모든 연산 기록하기
- 왜?
1. 테이프는 정방향 패스에 기록할 연산을 알아야 함
2. 테이프는 중간 출력에 대한 참조를 보유함 : 불필요한 연산을 기록하지 않음
3. 가장 일반적인 용례는 모든 모델의 훈련 가능한 변수에 대한 손실의 그래디언트 계산임

In [ ]:
# 훈련 가능
x0 = tf.Variable(3., name = 'x0')

# 훈련 불가능
x1 = tf.Variable(3., name = 'x1', trainable = False)

# Variable이 아님 (Varaible + Tensor = Tensor)
x2 = tf.Variable(2., name = 'x2') + 1.0

# Variable이 아님
x3 = tf.constant(3., name = 'x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1 ** 2) + (x2 ** 2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g) # Variable인 x0에 대한 그래디언트만 계산되었고, 나머지(trainable = False, Tensor, Constant(=Tensor)에 대한 값은 계산되지 않아 None 출력)

In [ ]:
# 테이프에서 감시 중인 변수
[var.name for var in tape.watched_variables()]

In [ ]:
x = tf.constant(3.) # Tensor

with tf.GradientTape() as tape:
  tape.watch(x) # Tensor에 대한 그래디언트를 기록하려면 watch 메소드가 필요하다.
  y = x**2

dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

In [ ]:
x0 = tf.Variable(0.)
x1 = tf.Variable(10.)

# 모든 Variable에 대한 기본 감시 비활성화 (watch_accessed_variables = False)
with tf.GradientTape(watch_accessed_variables = False) as tape:
  tape.watch(x1) # watch를 켜면 위에서 비활성화했어도 활성화됨
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

In [ ]:
grad = tape.gradient(ys, {'x0' : x0, 'x1' : x1})
print(grad['x0'], grad['x1']) # None, Tensor

## 중간 결과

In [ ]:
x = tf.constant(3.)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = x * x
  z = y * y

tape.gradient(z, y).numpy() # dz / dy = 2y = 2x**2 = 18

### GradientTape().gradient
- 호출되면 GradientTape의 리소스가 해제된다.
- 같은 계산에 대해 여러 그래디언트를 계산하려면 `persistent = True`인 그래디언트 테이프를 만든다.
- 계산 후에는 `del tape` 해주면 됨

In [ ]:
x = tf.constant([1, 3.]) # 이 경우 type은 float32가 됨
with tf.GradientTape(persistent = True) as tape:
  tape.watch(x)
  y = x * x
  z = y * y

print(tape.gradient(z, x).numpy()) # dz_dx = 4x**3 = 3, 108
print(tape.gradient(y, x).numpy()) # dy_dx = 2x = 2, 6

### 성능 참고 사항
- 테이프 컨텍스트 내에서 연산 수행하는 것에 대해 작은 오버헤드가 있다. 필요한 경우에만 `tape`를 사용할 것
- 그래디언트 테이프는 입출력을 포함한 중간 결과를 저장한다. 효율성을 위해 일부 연산(`ReLU` 등)은 중간 결과를 유지할 필요가 없고, 이는 정방향 패스 동안 정리된다.

### 그래디언트는 기본적으로 스칼라에 대한 연산이다.
- 여러 대상의 그래디언트를 요청하면? - 각 대상의 그래디언트의 합계가 나옴

In [ ]:
x = tf.Variable(2.)
with tf.GradientTape() as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient({'y0' : y0, 'y1' : y1}, x).numpy()) # dy0_dx1, dy1_dx 가 아니라 이 둘의 합계가 나옴

In [ ]:
x = tf.Variable(2.)

with tf.GradientTape() as tape:
  y = x * [3., 4.]

print(tape.gradient(y, x).numpy()) # 마찬가지로 그래디언트 값이 행렬이지만, 결과는 각 원소의 합계가 나온다.

- 별도의 그래디언트가 필요하다면 Jakovian을 참고하라고 함. 나중에 고급 AutoDiff에서 나옴

In [ ]:
x = tf.linspace(-10., 10., 200 + 1)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.nn.sigmoid(x) # 시그모이드 자체

dy_dx = tape.gradient(y, x) # 시그모이드를 다시 x로 미분한 값

plt.plot(x, y, label = 'y')
plt.plot(x, dy_dx, label = 'dy_dx')
plt.legend()

_ = plt.xlabel('x')

## 흐름 제어하기
-  Python의 문법이 자연스럽게 처리됨

In [ ]:
x = tf.constant(1.)

v0 = tf.Variable(x)
v1 = tf.Variable(2.)

with tf.GradientTape(persistent = True) as tape:
  tape.watch(x)

  # 이 경우 result 값은 if 조건이 True인 곳에 대해서만 값이 저장된다.
  if x > 0.:
    result = v0
  else:
    result = v1 ** 2

dv0, dv1 = tape.gradient(result, [v0, v1])

print(dv0, dv1)

In [ ]:
dx = tape.gradient(result, x)
print(dx) # x값에 따라 result는 v0나 v1**2이다. 그러나 result는 x에 관한 식이 아님 -> None이 나올 수 밖에

### None의 Gradient 구하기

In [ ]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape(persistent = True) as tape:
  z = y * y

print(tape.gradient(z, x)) # x는 tape에 기록되지 않음

# persistent = True가 없다면 아래 코드는 실행되지 않음 : 위의 gradient 식으로 인해 tape에 기록된 값들이 날아갔기 때문에
print(tape.gradient(z, y))

- 그래디언트의 연결이 끊어지는 사유들이 있다.

1. `Variables.assign_add()` 대신 `Variables` 객체에 Operator를 가해서 `Tensor`로 바뀌는 경우

In [ ]:
x = tf.Variable(2.)

for epoch in range(2):
  with tf.GradientTape() as tape:
    y = x + 1

  print(type(x).__name__, " : ", tape.gradient(y, x))
  # x = x + 1 # Variable에 스칼라 연산을 가하면 Tensor가 됨 - 그래디언트가 끊김
  x.assign_add(1) # Variable 업데이트는 해당 메소드를 이용한다.


2. `Tensorflow` 외부에서 계산된 경우

In [ ]:
x = tf.Variable([[1., 2.],
                 [3., 4.]], dtype = tf.float32)

with tf.GradientTape() as tape:
  x2 = x**2

  # Tensorflow가 아닌 Numpy로 계산되었음에 주목
  y = np.mean(x2, axis=0)

  # y는 array이기 때문에 텐서플로우의 연산을 적용할 수 없다.
  y = tf.reduce_mean(y, axis = 0)

  # 아래처럼 텐서로 바꾼 뒤 연산을 적용할 것
  # y = tf.convert_to_tensor(y)
  # y = tf.reduce_mean(x2, axis = 0)

print(tape.gradient(y, x))

3. int, string을 통해 gradient를 구한 경우
- 그래디언트 계산은 `float`만 쓰인다는 것만 짚어두자

In [ ]:
x = tf.constant(10)

with tf.GradientTape() as g:
  g.watch(x)
  y = x * x

print(g.gradient(y, x)) # 그래디언트 누락 대신 유형 오류가 발생함

4. 상태 저장 개체로 그래디언트를 구함
- 테이프는 현재 상태만 볼 수 있다. 현재 상태에 이르게 된 기록은 볼 수 없다.
- `tf.Tensor` : 텐서가 작성된 후에는 변경할 수 없다. 값은 있지만 상태는 없다. 여태까지의 모든 연산은 상태 비저장임
- `tf.Variable` : 내부 상태 & 값을 갖는다. 변수와 관련한 그래디언트를 계산하는 게 일반적이나, 변수의 상태는 그래디언트 계산이 더 멀리 돌아가지 않도록 차단한다.

In [ ]:
x0 = tf.Variable(3.)
x1 = tf.Variable(3.)

with tf.GradientTape() as tape:
  x1.assign_add(x0) # x1에 x1 + x0이 할당됨. 별도의 = 필요 없나봄?

  # y는 (x1 + x0) ** 2 부터 기록이 시작됨 - x1부터 기록되지 않음
  y = x1 ** 2

print(tape.gradient(y, x0)) # 예상 : dy_dx0 = 2(x1 + x0)
                            # 실제 : dy_dx0 = None

## 그래디언트가 등록되지 않는 경우
- 일부 `tf.Operation`은 미분 불가능한 것으로 등록되어 `None`을 반환함.
- `tf.raw_ops` 페이지에는 그래디언트가 등록된 저수준 연산이 표시됨
- 그래디언트가 등록되지 않은 연산으로 미분하고 싶다면, 그래디언트를 구현하고 등록(`tf.RegisterGradient`)하거나 다른 ops로 함수를 다시 구현해야 함

In [ ]:
image = tf.Variable([[[.5, .0, .0]]])
delta = tf.Variable(.1)

with tf.GradientTape() as tape:
  new_image = tf.image.adjust_contrast(image, delta) #그래디언트를 가질 수 있음

try:
  print(tape.gradient(new_image, [image, delta])) # 구현되지 않은 raw_ops.AdjustContrastV2를 반환함
  assert False 
except LookupError as e: # 그래디언트가 등록되지 않은 float op -> 오류 발생시킴
  print(f"{type(e).__name__} : {e}")

### 연결되지 않은 그래디언트 : None 대신 0

In [ ]:
x = tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y**2

print(tape.gradient(z, x, unconnected_gradients = tf.UnconnectedGradients.ZERO)) # dz_dx는 None이 떠야 하지만..